## Import and load the data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

## Preprocess data

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

56 documents
13 classes ['age', 'goodbye', 'greeting', 'happy', 'hate', 'joke', 'laugh', 'love', 'name', 'options', 'sad', 'story', 'thanks']
75 unique lemmatized words ["'s", ',', 'a', 'age', 'am', 'anyone', 'are', 'awesome', 'be', 'bored', 'bye', 'call', 'can', 'chatting', 'cool', 'could', 'day', 'do', 'down', 'everyone', 'for', 'fun', 'funny', 'good', 'goodbye', 'great', 'ha', 'haha', 'happy', 'hate', 'hehe', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'joke', 'know', 'later', 'lol', 'love', 'may', 'me', 'myself', 'name', 'next', 'nice', 'offered', 'old', 'please', 'provide', 'read', 'sad', 'see', 'story', 'support', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'wonderful', 'yay', 'you', 'your', 'yourself']


## Create training and testing data

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Rakshu\AppData\Local\Temp/ipykernel_17676/3528693901.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Build the model 

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200


C:\Users\Rakshu\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


12/12 [==============================] - 1s 3ms/step - loss: 2.5911 - accuracy: 0.0714
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 2.5076 - accuracy: 0.0893
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 2.4796 - accuracy: 0.1786
Epoch 4/200
12/12 [==============================] - 0s 3ms/step - loss: 2.5211 - accuracy: 0.0536
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 2.3809 - accuracy: 0.2143
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 2.3701 - accuracy: 0.2500
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 2.2668 - accuracy: 0.3214
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1766 - accuracy: 0.3214
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 2.0837 - accuracy: 0.3571
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 2.0014 - accuracy: 0.3750
Epoch 11/200
12/12 [=

12/12 [==============================] - 0s 3ms/step - loss: 0.0936 - accuracy: 0.9821
Epoch 84/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2118 - accuracy: 0.9464
Epoch 85/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1759 - accuracy: 0.9107
Epoch 86/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1814 - accuracy: 0.9643
Epoch 87/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1599 - accuracy: 0.9643
Epoch 88/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1180 - accuracy: 0.9643
Epoch 89/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2721 - accuracy: 0.8929
Epoch 90/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1054 - accuracy: 0.9821
Epoch 91/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0976 - accuracy: 0.9643
Epoch 92/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1175 - accuracy: 0.9821
Epoch 93/200


12/12 [==============================] - 0s 2ms/step - loss: 0.0947 - accuracy: 0.9643
Epoch 165/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0540 - accuracy: 0.9821
Epoch 166/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0765 - accuracy: 0.9821
Epoch 167/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0456 - accuracy: 0.9821
Epoch 168/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 169/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0288 - accuracy: 1.0000
Epoch 170/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0301 - accuracy: 1.0000
Epoch 171/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0544 - accuracy: 0.9821
Epoch 172/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0200 - accuracy: 1.0000
Epoch 173/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0424 - accuracy: 1.0000
Epoc